In [2]:
# Few-shot learning is a technique that allows you to train a model on a small amount of data.
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

chat = ChatOpenAI(
	model_name="gpt-3.5-turbo",
	temperature=0.1, 
	streaming=True, 
	callbacks=[StreamingStdOutCallbackHandler()]
)
	

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]		

example_prompt = """
  Human: {question}
	AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_prompt)

prompt = FewShotPromptTemplate(
	example_prompt=example_prompt, 
	examples=examples,
	suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({
    "country": "Finland"
})

AI: 
        Here is what I know:
        
        Capital: Helsinki
        Language: Finnish, Swedish
        Food: Rye bread, fish, berries
        Currency: Euro

AIMessageChunk(content='AI: \n        Here is what I know:\n        \n        Capital: Helsinki\n        Language: Finnish, Swedish\n        Food: Rye bread, fish, berries\n        Currency: Euro')

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

examples = [
    {
        "country": "France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]		

example_prompt = ChatPromptTemplate.from_messages([
	("human", "What do you know about {country}?"),
	("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
	example_prompt=example_prompt,
	examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
	("system", "You are a geography expert."),
	example_prompt, # Put the few-shot prompt here
	("human", "What do you know about {country}?"),
])

chain = final_prompt | chat

chain.invoke({"country": "Turkey"})


        I know this:
        Capital: Ankara
        Language: Turkish
        Food: Kebabs and Baklava
        Currency: Turkish Lira
        

AIMessageChunk(content='\n        I know this:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebabs and Baklava\n        Currency: Turkish Lira\n        ')

In [9]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]		

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=180,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Brazil")


'Human: What do you know about France?\nAI: \n        Here is what I know:\n        \n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Italy?\nAI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [14]:
from langchain.prompts.example_selector.base import BaseExampleSelector

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)
        
    def select_examples(self, input_examples):
        from random import choice
        return [choice(self.examples)]

example_selector = RandomExampleSelector(examples=examples)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Brazil")

'Human: What do you know about Italy?\nAI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'